# `CSV -> Processing -> LSTM Model Training`

# 1. Imports and directories

In [2]:
!pip install mediapipe==0.8.9.1 --quiet
!pip install tensorflow --quiet

In [3]:
import os
import cv2
# from google.colab.patches import cv2_imshow ## NOT FOR JUPYTER NOTEBOOKS
import mediapipe as mp
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

import tensorflow
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Masking
from sklearn.metrics import confusion_matrix, accuracy_score

2023-09-03 16:31:17.209343: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 16:31:17.822067: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 16:31:17.828679: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 16:31:20.114741: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
DIRECTORY_PATH = "videos_fold_1"
CSV_DIRECTORY = "csv_data_output" # NOTE this folder will be nested inside DIRECTORY_PATH
PLOT_PNG_DIRECTORY = "png_plots"
NO_NANS_CSV_DIRECTORY = "csv_data_no_nans" # this is where the no nan value csv files will go.
NO_NANS_TO_PLOT = "csv_data_no_nans_plots"
CSV_FOR_MODEL = "csv_data_for_model" # this is basically like making a backup of original data to work on, just in case of accidental modification.
LOG_PATH = "tensorflow_logs"

DIRECTORY_TO_CSV = f"{DIRECTORY_PATH}/{CSV_DIRECTORY}"
DIRECTORY_TO_PNG = f"{DIRECTORY_PATH}/{PLOT_PNG_DIRECTORY}"
DIRECTORY_TO_NO_NANS_CSV = f"{DIRECTORY_PATH}/{NO_NANS_CSV_DIRECTORY}"
DIRECTORY_TO_CSV_FOR_MODEL = f"{DIRECTORY_PATH}/{CSV_FOR_MODEL}"
DIRECTORY_TO_NO_NANS_TO_PLOT = f"{DIRECTORY_PATH}/{NO_NANS_TO_PLOT}"
DIRECTORY_TO_LOG = f"{DIRECTORY_PATH}/{NO_NANS_CSV_DIRECTORY}/{LOG_PATH}"


# ================ MOVE THIS CLOSER TO WHERE IT WILL BE USED? ==================# 
PAD_SCALAR = 1000
VIDEO_TIME_MINS = 10
TRIM_TIME = 10 # Seconds
FPS_GUESS = 30
TRIM_ROWS = TRIM_TIME * FPS_GUESS # This is the number of rows that will be trimmed from the beginning and end of the data.
MAX_ROWS = FPS_GUESS*VIDEO_TIME_MINS*60 # video time in frames, probably 10mins at 30fps so 18,000.
# print(f"All pre-processed dataframes set to {MAX_ROWS} rows before trimming. This means padding files where needed.\nThen be trimmed by {TRIM_ROWS} from the beginning and end.")

# 2. Data Pre-Processing

## 2.1. (CAN SKIP) First, make a copy of the original csv files for backup (SHOULD ALREADY BE DONE)

In [5]:
if os.path.exists(DIRECTORY_TO_CSV_FOR_MODEL):
    print(f">> Oh no, the path ~/{DIRECTORY_TO_CSV_FOR_MODEL} already exists.\n>>>> Maybe you already made the csv files?")
else:
    print(f">> ~/{DIRECTORY_TO_CSV_FOR_MODEL} doesn't exist.\n>>>> Creating and copying files now...")
    shutil.copytree(DIRECTORY_TO_CSV, DIRECTORY_TO_CSV_FOR_MODEL)
    print(f">> Successfully copied directory {DIRECTORY_TO_CSV} to ~/{DIRECTORY_TO_CSV_FOR_MODEL}.\n")
    print(">>>> CONFIRM YOUR FOLDER. THERE COULD BE CACHED FILES COPIED IN IT.")

>> Oh no, the path ~/videos_fold_1/csv_data_for_model already exists.
>>>> Maybe you already made the csv files?


## 2.2. (CAN SKIP) Make a list of csv files, confirm all your data is present

In [6]:
def generate_list_of_csv(directory):
    """
    Takes a directory with csv files then generates a list containing their directory paths that can be iterated over.
    """
    csv_list = []  
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file has a video extension (you can add more extensions)
            if file.lower().endswith(('.csv')):
                csv_path_and_filename = os.path.join(root, file)
                csv_list.append(csv_path_and_filename)
    return csv_list

# Generate a list of all CSVs for ALL CSVs in the directory
list_of_csv = generate_list_of_csv(DIRECTORY_TO_CSV_FOR_MODEL)
print(f"\nThere are {len(list_of_csv)} csv files in your list from directory '{DIRECTORY_TO_CSV_FOR_MODEL}'")


There are 144 csv files in your list from directory 'videos_fold_1/csv_data_for_model'


## 2.3. Seperate CSV files by class and store in List

In [7]:
def generate_csv_class_lists(directory):
    
    # Initialize lists to hold the paths of CSV files for each class 
    alert = []
    normal= []
    drowsy = []   
    
    # Iterate over the files in the CSV folder
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            if "class_0" in filename:  # Using filenames containing 'class_0' for alert videos
                alert.append(filepath)
            elif "class_5" in filename:  # Using filenames containing 'class_5' for normal videos
                normal.append(filepath)
            elif "class_10" in filename:  # Using filenames containing 'class_10' for drowsy videos
                drowsy.append(filepath)

    alert.sort(reverse=False)
    normal.sort(reverse=False)
    drowsy.sort(reverse=False)

    return alert, normal, drowsy

alert_csv_file_list, normal_csv_file_list, drowsy_csv_file_list = generate_csv_class_lists(DIRECTORY_TO_CSV_FOR_MODEL)

# Verify sorting:
print(f"Alert csv files: {len(alert_csv_file_list)}")
for alert_csv in alert_csv_file_list:
    print(alert_csv)
print("")    
print(f"Normal csv files: {len(normal_csv_file_list)}")
for normal_csv in normal_csv_file_list:
    print(normal_csv)
print("")      
print(f"Drowsy csv files: {len(drowsy_csv_file_list)}")
for drowsy_csv in drowsy_csv_file_list:
    print(drowsy_csv)
print("")   

Alert csv files: 48
videos_fold_1/csv_data_for_model/data_person_10_class_0.csv
videos_fold_1/csv_data_for_model/data_person_11_class_0.csv
videos_fold_1/csv_data_for_model/data_person_12_class_0.csv
videos_fold_1/csv_data_for_model/data_person_13_class_0.csv
videos_fold_1/csv_data_for_model/data_person_14_class_0.csv
videos_fold_1/csv_data_for_model/data_person_15_class_0.csv
videos_fold_1/csv_data_for_model/data_person_16_class_0.csv
videos_fold_1/csv_data_for_model/data_person_17_class_0.csv
videos_fold_1/csv_data_for_model/data_person_18_class_0.csv
videos_fold_1/csv_data_for_model/data_person_19_class_0.csv
videos_fold_1/csv_data_for_model/data_person_1_class_0.csv
videos_fold_1/csv_data_for_model/data_person_20_class_0.csv
videos_fold_1/csv_data_for_model/data_person_21_class_0.csv
videos_fold_1/csv_data_for_model/data_person_22_class_0.csv
videos_fold_1/csv_data_for_model/data_person_23_class_0.csv
videos_fold_1/csv_data_for_model/data_person_24_class_0.csv
videos_fold_1/csv_dat

## 2.4. Generate lists of Dataframes. 
#### `A: Remove Time col`
#### `B: Trim frames (Including reset_index!)`
#### `C: Impute mean`
#### `D: Return lists of classes with modified ti (trim-imputed) dataframes`
### Run it for each class!

In [8]:
def load_csv_to_df_trim_impute(list_of_csv_class, MAX_ROWS, TRIM_ROWS, IMPUTE="mean"):
    """
    This function takes a list of csv filepaths and converts the csvs to dataframes, with NO TIME COLUMN, TRIM, and IMPUTE.
    RETURNS a list of processed dataframes.
    
    Note that this function operates on the frame index. 
    If you want to smooth data in a standardised way, switch index to time column, copy the code to a new function and code it from there.
    """
    # Columns to keep in the dataframes.
    df_list_of_measured_cols = [
    'left_eye_aperture_measurements',
    'right_eye_aperture_measurements', 
    'mouth_top_bottom_aperture_measurements',
    'mouth_left_right_aperture_measurements'
    ]
    
    list_of_csv = list_of_csv_class
    trimmed_imputed_dfs = []
    number_of_files_with_nans = 0
    total_nans = 0


    # Generate Dataframe with only specified columns (no time)
    for csv_file in list_of_csv:
        df = pd.read_csv(csv_file, usecols=df_list_of_measured_cols)
        # check shape of df
        print(f"{csv_file}:    {df.shape}")
        
        # Trim Dataframe - First, trim the df AND reset index.
        dft = df.iloc[TRIM_ROWS:-TRIM_ROWS]
        dft.reset_index(drop=True, inplace=True)
        # If the df is still longer than max_rows, slice it from 0 to max_rows. This maintains index starting from zero.
        if len(dft) > MAX_ROWS: 
            dft = dft.iloc[0:MAX_ROWS]
            print(f"{csv_file} loaded to dataframe successfully with a max_rows trim")
        else:
            print(f"{csv_file} loaded to dataframe successfully")

        # Check shape of df and dft
        print(f"{csv_file}:    {dft.shape}")
        
        # If dft has NaNs, then impute it 
        number_of_nans = dft.isna().sum().sum()
        total_nans += number_of_nans
        
        if number_of_nans > 0:
            number_of_files_with_nans += 1
            print(f">> Has {number_of_nans} NaNs. Preparing to impute...")
            # iterate over each column in the df to impute nans.
            for column in df_list_of_measured_cols:
                # Instantiate a SimpleImputer object with your strategy of choice
                imputer = SimpleImputer(strategy="mean") 
                # Call the "fit" method on the object 
                imputer.fit(dft[[column]])
                # Call the "transform" method on the object
                dft.loc[:, column] = imputer.transform(dft[[column]])
                # print(f">>>> '{column}' column imputed.")
            print(f">> All columns imputed.")
        else:
            print(f">> NO NaNs. Skipping file with no modifications.")
        # Appending dft to trummed_imputed_dfs.
        trimmed_imputed_dfs.append(dft)
        print(f">>>> Success! {csv_file} trimmed and imputed as df and appended.\n")
        
    print(f">>>>>>>> Succssfully trimmed all files. {number_of_files_with_nans} files had {total_nans} total NaNs and were imputed\n")
   
    # return dataframes to a list
    return trimmed_imputed_dfs


list_of_trimputed_alert_dfs = load_csv_to_df_trim_impute(alert_csv_file_list, MAX_ROWS, TRIM_ROWS, IMPUTE="mean")
list_of_trimputed_normal_dfs = load_csv_to_df_trim_impute(normal_csv_file_list, MAX_ROWS, TRIM_ROWS, IMPUTE="mean")
list_of_trimputed_drowsy_dfs = load_csv_to_df_trim_impute(drowsy_csv_file_list, MAX_ROWS, TRIM_ROWS, IMPUTE="mean")

videos_fold_1/csv_data_for_model/data_person_10_class_0.csv:    (16309, 4)
videos_fold_1/csv_data_for_model/data_person_10_class_0.csv loaded to dataframe successfully
videos_fold_1/csv_data_for_model/data_person_10_class_0.csv:    (15709, 4)
>> NO NaNs. Skipping file with no modifications.
>>>> Success! videos_fold_1/csv_data_for_model/data_person_10_class_0.csv trimmed and imputed as df and appended.

videos_fold_1/csv_data_for_model/data_person_11_class_0.csv:    (18818, 4)
videos_fold_1/csv_data_for_model/data_person_11_class_0.csv loaded to dataframe successfully with a max_rows trim
videos_fold_1/csv_data_for_model/data_person_11_class_0.csv:    (18000, 4)
>> NO NaNs. Skipping file with no modifications.
>>>> Success! videos_fold_1/csv_data_for_model/data_person_11_class_0.csv trimmed and imputed as df and appended.

videos_fold_1/csv_data_for_model/data_person_12_class_0.csv:    (16919, 4)
videos_fold_1/csv_data_for_model/data_person_12_class_0.csv loaded to dataframe successful

## 2.5. Train, Test, Split on Combined Data `RANDOM IS OFF`

In [9]:
# Setting up Binary Classification split, so only combinind 2 classes - drowsy and alert.
# binary_data is a list of dataframes for drowsy and alert.
binary_data = list_of_trimputed_drowsy_dfs + list_of_trimputed_alert_dfs
# labels_binary_data is a list of numbers that represent the class of the dataframes. 1 for drowsy, and 0 for alert.
labels_binary_data = [1] * len(list_of_trimputed_drowsy_dfs) + [0] * len(list_of_trimputed_alert_dfs)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(binary_data, labels_binary_data, test_size=0.25) #, random_state=42)

# Verification
print(binary_data)
print("")
print(labels_binary_data)
print("X_split: training set:", len(X_train), ". test set:", len(X_test))
print("y_split: training set:", len(y_train), ". test set:", len(y_test))

[       left_eye_aperture_measurements  right_eye_aperture_measurements  \
0                            0.011029                         0.010825   
1                            0.010958                         0.010784   
2                            0.011176                         0.010880   
3                            0.011686                         0.011331   
4                            0.011714                         0.011467   
...                               ...                              ...   
17995                        0.012754                         0.012456   
17996                        0.013433                         0.013134   
17997                        0.013356                         0.013074   
17998                        0.013702                         0.013451   
17999                        0.013597                         0.013364   

       mouth_top_bottom_aperture_measurements  \
0                                    0.024706   
1           

## 2.6.1. Normalizing the Data

In [10]:
def normalize_train_test_datasets(list_of_training_dfs, list_of_test_dfs):
    """
    It is important not to have data leakage. 
    So, I am going to fit_transform the normaliser on the ENTIRE training dataset and then transform the ENTIRE test dataset.
    """
    # Initialize preprocessing steps (e.g., StandardScaler)
    scaler = StandardScaler()

    # Let's keep the column names! This is already defined in "df_list_of_measured_cols" in 2.4.
    column_names = list_of_training_dfs[0].columns
    
    # Calculate preprocessing parameters on ALL training data
    combined_train_data = pd.concat(list_of_training_dfs)  # Combine all dataframes into one for training
    scaler.fit(combined_train_data)
    
    # Apply preprocessing to training and testing data
    preprocessed_train_data = [scaler.transform(df) for df in list_of_training_dfs]
    preprocessed_test_data = [scaler.transform(df) for df in list_of_test_dfs] 
    
    # Convert preprocessed arrays back to DataFrames with column names
    preprocessed_train_data_dfs = [pd.DataFrame(data=arr, columns=column_names) for arr in preprocessed_train_data]
    preprocessed_test_data_dfs = [pd.DataFrame(data=arr, columns=column_names) for arr in preprocessed_test_data]

    # calculate the minimum value of of the training and test sets.
    training_set_minimum_value = np.min(np.vstack(preprocessed_train_data))
    test_set_minimum_value = np.min(np.vstack(preprocessed_test_data))

    return preprocessed_train_data_dfs, preprocessed_test_data_dfs, training_set_minimum_value, test_set_minimum_value


X_train_normalized_dfs, X_test_normalized_dfs, X_train_min_value, X_test_min_value = normalize_train_test_datasets(X_train, X_test)
print(X_train_min_value, X_test_min_value)

-2.639340578233545 -2.3608847636634054


In [11]:
# Verification
# print(X_train_normalized)
# print("")
# print(X_test_normalized)
# print(X_train_normalized_dfs[4])
# print(X_train_normalized_dfs[4].shape)

print(X_test_normalized_dfs[11])
print(X_test_normalized_dfs[11].shape)

       left_eye_aperture_measurements  right_eye_aperture_measurements  \
0                           -0.376988                        -0.430452   
1                           -0.246583                        -0.301198   
2                           -0.277044                        -0.329507   
3                           -0.238209                        -0.298394   
4                           -0.176745                        -0.233142   
...                               ...                              ...   
10109                       -0.247045                        -0.340241   
10110                       -0.207788                        -0.298288   
10111                       -0.203568                        -0.309219   
10112                       -0.210823                        -0.308016   
10113                       -0.207757                        -0.303971   

       mouth_top_bottom_aperture_measurements  \
0                                    0.323832   
1            

## 2.6.2. Determining the min value for padding

In [12]:
def get_padding_value(X_train_min_value, X_test_min_value, scale_factor):
    """
    Returns the padding value depending on which X_train or X_test min is smaller, and multiplies it by a scale_factor.
    """
    if X_train_min_value < X_test_min_value: # train is smaller
        padding_value = X_train_min_value
        
    elif X_train_min_value > X_test_min_value: # test is smaller
        padding_value = X_test_min_value
        
    else:                                      # If they're equal!
        padding_value = X_train_min_value

    return padding_value * scale_factor

# test it out:
int(get_padding_value(X_train_min_value, X_test_min_value, PAD_SCALAR))

-2639

## 2.7. Pad the Data (no imputing, just adjusting the length of the dataframes)
#### If you use this for imputing NaNs, make sure you comment out the impute code in 2.4 part C!
#### `NOTE: This function "pad_dataframes" returns NUMPY ARRAYS`

In [13]:
def pad_dataframes(list_of_dataframes, max_sequence_length: int, padding_value):
    """
    This function will loop through a list of dataframes. For each dataframe, it will determine if the dataframe needs padding
    if its length is less than the max_sequence_length. If so, it will be padded with a padding value. If not, it will not be padded.
    The padding value will be calculated as a value smaller than the mimimum value in ALL dataframes, calculated after the normalisation.
    THIS FUNCTION RETURNS NUMPY ARRAYS.
    """
    padded_dataframes = []
    padded_rows_sum = 0

    for df in list_of_dataframes:
        print("df.shape:", df.shape)
        if len(df) < max_sequence_length:
            # Pad each column with the specified padding value up to max_sequence_length rows
            padded_df = df.copy()
            num_rows_to_pad = max_sequence_length - len(padded_df)
            print(f">> Padding {num_rows_to_pad} rows in this dataframe.")
            padding_values = np.full((num_rows_to_pad, df.shape[1]), padding_value)  # Create padding values
            
            # print(padding_values)
            
            padded_df = np.vstack((padded_df, padding_values))  # Stack original data and padding
            padded_dataframes.append(padded_df)

            padded_rows_sum += num_rows_to_pad
        else:
            print(">> No padding required.")
            padded_dataframes.append(df.values)                 # Convert the DataFrame to a numpy array and append

    print(f"\n>>>> Padding Completed. {padded_rows_sum} rows were successfully padded.\n")

    return padded_dataframes


padding_value = int(get_padding_value(X_train_min_value, X_test_min_value, scale_factor=PAD_SCALAR))

print("     X_train_normalized_padded")
X_train_normalized_padded = pad_dataframes(X_train_normalized_dfs, MAX_ROWS, padding_value)

print("     X_test_normalized_padded")
X_test_normalized_padded = pad_dataframes(X_test_normalized_dfs, MAX_ROWS, padding_value)

     X_train_normalized_padded
df.shape: (17740, 4)
>> Padding 260 rows in this dataframe.
df.shape: (17416, 4)
>> Padding 584 rows in this dataframe.
df.shape: (8650, 4)
>> Padding 9350 rows in this dataframe.
df.shape: (17173, 4)
>> Padding 827 rows in this dataframe.
df.shape: (17387, 4)
>> Padding 613 rows in this dataframe.
df.shape: (17427, 4)
>> Padding 573 rows in this dataframe.
df.shape: (17788, 4)
>> Padding 212 rows in this dataframe.
df.shape: (9633, 4)
>> Padding 8367 rows in this dataframe.
df.shape: (17524, 4)
>> Padding 476 rows in this dataframe.
df.shape: (15709, 4)
>> Padding 2291 rows in this dataframe.
df.shape: (17450, 4)
>> Padding 550 rows in this dataframe.
df.shape: (17810, 4)
>> Padding 190 rows in this dataframe.
df.shape: (17456, 4)
>> Padding 544 rows in this dataframe.
df.shape: (17030, 4)
>> Padding 970 rows in this dataframe.
df.shape: (17621, 4)
>> Padding 379 rows in this dataframe.
df.shape: (18000, 4)
>> No padding required.
df.shape: (14498, 4)
>>

In [14]:
# Verify:
print(X_train_normalized_padded[0])
print("")
print(X_test_normalized_padded[0])
print("")

[[-9.60713413e-01 -9.17031887e-01  8.21460998e-02  4.74837197e-01]
 [-9.21196698e-01 -8.70085430e-01  1.24031276e-01  4.93153586e-01]
 [-1.03826149e+00 -1.00089952e+00  1.54047836e-01  5.21752431e-01]
 ...
 [-2.63900000e+03 -2.63900000e+03 -2.63900000e+03 -2.63900000e+03]
 [-2.63900000e+03 -2.63900000e+03 -2.63900000e+03 -2.63900000e+03]
 [-2.63900000e+03 -2.63900000e+03 -2.63900000e+03 -2.63900000e+03]]

[[-2.55755528e-01 -4.75425301e-01 -5.74390357e-01  1.24879557e-01]
 [-3.13136857e-01 -5.09319778e-01 -5.52000736e-01  1.38589261e-01]
 [-5.13510383e-01 -6.80358856e-01 -5.05536595e-01  1.49758795e-01]
 ...
 [-2.63900000e+03 -2.63900000e+03 -2.63900000e+03 -2.63900000e+03]
 [-2.63900000e+03 -2.63900000e+03 -2.63900000e+03 -2.63900000e+03]
 [-2.63900000e+03 -2.63900000e+03 -2.63900000e+03 -2.63900000e+03]]



In [15]:
# more Verify:
for arr in X_train_normalized_padded:
    print(arr.shape)
print("")
for arr in X_test_normalized_padded:
    print(arr.shape)

(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)

(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4)
(18000, 4

## 2.8. Divide the Feature and Target arrays into splits

In [16]:
def split_numpy_arrays(x_set, y_set, row_splits: int):
    """
    ** NOTE: as of last updated code, this function actually takes the x_set input of a list of numpy arrays.
    
    Pass in a list of corresponding dataframes (x_set) and target labels (y_set), e.g. X_train and y_train, equal length lists.
    Function will split each df inside the X_set, and create the same number of targets as the number of splits.
    This function assumes all dataframes inside the x_set are the same length i.e. same number of rows.
    e.g. a df with 500 rows and row_chunks 50 will return 10 dataframes of length 50 each.
    !!!!!! Only pass in values that will result in an integer ratio of len(df) // row_chunks!!!!!
    Returns the new split df X_set and y_set to pass through to the model.
    """
    len_x = len(x_set)
    len_y = len(y_set)

    if len_x == len_y and len_x != 0:                             # This will only run if the lists contain data and are the same length.
        original_array_length = x_set[0].shape[0]                 # 18,000 rows
        num_splits = original_array_length // row_splits          # 18,000 rows / 150 rows (so about 5 seconds of frame data 30fps per chunk) = 150 chunks
        print(f">> Performed {num_splits} splits on {len_x} arrays with {row_splits} rows each.")

        list_of_split_arrays = []
        list_of_targets_expanded = []

        # For each array in x_set
        for i in range(len_x):
            # get the array and corresponding target
            array = x_set[i]                                       # gets the array in the i position in x_set
            target_to_expand = y_set[i]                            # gets the target value in the i position in y_set, corresponding to x_set.

            split_count = 0
            # For each split we're creating from the array in x_set
            for j in range(num_splits):
                # define the start and end indexes
                start_idx = split_count * row_splits              
                end_idx = (split_count + 1) * row_splits
                # slice the input array from the start index to end index
                split_array = array[start_idx:end_idx]
                # append this slice to the list of split arrays.
                list_of_split_arrays.append(split_array)
                # append the corresponding target value.
                list_of_targets_expanded.append(target_to_expand)
                split_count += 1
    else:
        print("You're x feature and y label sets are not the same length. Fix that problem and come back")

    return list_of_split_arrays, list_of_targets_expanded


X_train_normalized_padded_splits, y_train_splits = split_numpy_arrays(X_train_normalized_padded, y_train, 300)
print(f"---- len(X_train_normalized_padded_splits) = {len(X_train_normalized_padded_splits)}. len(y_train_splits) = {len(y_train_splits)}.")

X_test_normalized_padded_splits, y_test_splits = split_numpy_arrays(X_test_normalized_padded, y_test, 300)
print(f"---- len(X_test_normalized_padded_splits) = {len(X_test_normalized_padded_splits)}. len(y_test_splits) = {len(y_test_splits)}.")

>> Performed 60 splits on 72 arrays with 300 rows each.
---- len(X_train_normalized_padded_splits) = 4320. len(y_train_splits) = 4320.
>> Performed 60 splits on 24 arrays with 300 rows each.
---- len(X_test_normalized_padded_splits) = 1440. len(y_test_splits) = 1440.


In [17]:
# verify
print(X_train_normalized_padded_splits[0])
print(X_test_normalized_padded_splits[0])

[[-0.96071341 -0.91703189  0.0821461   0.4748372 ]
 [-0.9211967  -0.87008543  0.12403128  0.49315359]
 [-1.03826149 -1.00089952  0.15404784  0.52175243]
 ...
 [-0.42307987 -0.35820105 -0.35336868  0.43677363]
 [-0.44097021 -0.3747751  -0.23136367  0.45484301]
 [-0.39113752 -0.33099865 -0.28650849  0.43439287]]
[[-0.25575553 -0.4754253  -0.57439036  0.12487956]
 [-0.31313686 -0.50931978 -0.55200074  0.13858926]
 [-0.51351038 -0.68035886 -0.50553659  0.14975879]
 ...
 [-1.27757871 -1.35468213 -0.62307553  0.17642588]
 [-1.27749692 -1.34963555 -0.64555749  0.19260745]
 [-1.24333471 -1.31605302 -0.67346101  0.16725904]]


## 2.9. Flatten the datasets
### Data needs to be "reshaped" before it can be passed in to model.

In [18]:
# Convert lists to numpy arrays
X_train_fit = np.array(X_train_normalized_padded_splits)
X_test_fit = np.array(X_test_normalized_padded_splits)
y_train_fit = np.array(y_train_splits)
y_test_fit = np.array(y_test_splits)

# Re-size the y output to match the dimensions of the input (a,b,c...)
y_train_fit_resized = y_train_fit.reshape(-1, 1)
y_test_fit_resized = y_test_fit.reshape(-1, 1)

# Check shapes
print(X_train_fit.shape)
print(X_test_fit.shape)
print(y_train_fit_resized.shape)
print(y_test_fit_resized.shape)

(4320, 300, 4)
(1440, 300, 4)
(4320, 1)
(1440, 1)


#### (optional) Plot the new graphs to see if they look OK. Just open and inspect is OK, too.

# 3. LSTM Model

In [19]:
# Define input shape (number of time steps and number of features)
input_shape = (X_train_normalized_padded_splits[0].shape[0], X_train_normalized_padded_splits[0].shape[1])
print("Input Shape:", input_shape)
# Define number of classes
num_classes = 1     # 1 for binary!
print(padding_value)

Input Shape: (300, 4)
-2639


## 3.1. Create the Architecture

In [20]:
def initialize_model(input_shape, num_classes, mask_value):
    """
    First try - relu activation, sparse_categorical_crossentropy, Adam, softmax.
    Second try - tahn activation, binary_crossentropy
    """
    # Set up a sequential model
    model = Sequential()
    # Catch and remove the padded data with a masking layer:
    model.add(Masking(mask_value=mask_value, input_shape=input_shape))
    # LSTM layer 1
    model.add(LSTM(units=64, return_sequences=True, activation='tanh'))
    model.add(Dropout(0.2))
    # LSTM layer 2
    model.add(LSTM(units=128, return_sequences=False, activation='tanh'))
    model.add(Dropout(0.2))
    
    # Dense layer
    model.add(Dense(64, activation='tanh'))
    # model.add(Dropout(0.1))
    # Dense layer
    model.add(Dense(32, activation='tanh'))
    
    # Output layer
    model.add(Dense(num_classes, activation='sigmoid'))
    
    return model


# Initialize the LSTM model
model = initialize_model(input_shape, num_classes, padding_value)

# Custom optimiser to tackle the loss nan problem:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.05)

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 300, 4)            0         
                                                                 
 lstm (LSTM)                 (None, 300, 64)           17664     
                                                                 
 dropout (Dropout)           (None, 300, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               98816     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 32)                2

## 3.2. Train the LSTM Model

In [21]:
## sess.graph contains the graph definition; that enables the Graph Visualizer.
file_writer = tensorflow.summary.create_file_writer(DIRECTORY_TO_LOG)

es = EarlyStopping(
    patience = 80,
    restore_best_weights = True
)

tb_callback = TensorBoard(
    log_dir=DIRECTORY_TO_LOG
)

history = model.fit(
    X_train_fit, 
    y_train_fit_resized, 
    batch_size=32, 
    epochs=1000,
    validation_data=(X_test_fit, y_test_fit_resized),
    callbacks = [es, tb_callback], 
    verbose=1
)

Epoch 1/1000
135/135 [==============================] - 82s 561ms/step - loss: 0.7652 - accuracy: 0.4995 - val_loss: 0.7408 - val_accuracy: 0.4986
Epoch 2/1000
135/135 [==============================] - 73s 538ms/step - loss: 0.7098 - accuracy: 0.5088 - val_loss: 0.7339 - val_accuracy: 0.4986
Epoch 3/1000
135/135 [==============================] - 71s 528ms/step - loss: 0.7150 - accuracy: 0.4898 - val_loss: 0.7155 - val_accuracy: 0.5000
Epoch 4/1000
135/135 [==============================] - 69s 512ms/step - loss: 0.7127 - accuracy: 0.4884 - val_loss: 0.7173 - val_accuracy: 0.5028
Epoch 5/1000
135/135 [==============================] - 73s 537ms/step - loss: 0.7114 - accuracy: 0.4928 - val_loss: 0.6969 - val_accuracy: 0.4972
Epoch 6/1000
135/135 [==============================] - 68s 504ms/step - loss: 0.7043 - accuracy: 0.5134 - val_loss: 0.7917 - val_accuracy: 0.5028
Epoch 7/1000
135/135 [==============================] - 66s 491ms/step - loss: 0.7196 - accuracy: 0.4998 - val_loss: 0

## 3.3. Evaluate The Model

y = model.predict(X_test)

In [ ]:
true_labels = [0, 1, 1, 1, 0, 0, 1, 1, 1, 0]
predicted_labels = [1, 1, 1, 0, 0, 1, 0, 1, 1, 0]

conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plotting the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
accuracy_score(true_labels, predicted_labels)

## 3.4. Saving the Weights

As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights.

In [ ]:
model.save(f"{DIRECTORY_TO_CSV_FOR_MODEL}/weight.h5")

## 3.5. Loading Model

### Re-initialise the model and compile it, THEN load the model:

In [ ]:
model.load(f"{DIRECTORY_TO_CSV_FOR_MODEL}/weight.h5")

# Real-Time Testing

In [ ]:
# Recording CV2 Framework

In [ ]:
# Get Reference Points from recording

In [ ]:
# When frame count = 150 (lstm input length)

# Appendix A: Array to Plot tool

In [1]:
def draw_data(array):
    """
    This function is designed to draw graphs. 
    Currently not saving them, but you can add that feature.
    """
    cols = [
    'left_eye_aperture_measurements',
    'right_eye_aperture_measurements', 
    'mouth_top_bottom_aperture_measurements',
    'mouth_left_right_aperture_measurements'
    ]
    
    df = pd.DataFrame(array, columns=cols)

    fig, ax = plt.subplots(figsize=(21,12))

    # Plotting apertures
    ax.plot(df['left_eye_aperture_measurements'], label='Left Eye Aperture', color='red')
    ax.plot(df['right_eye_aperture_measurements'], label='Right Eye Aperture', color='blue')
    ax.plot(df['mouth_top_bottom_aperture_measurements'], label='Mouth Top-Bottom Aperture', color='green')
    ax.plot(df['mouth_left_right_aperture_measurements'], label='Mouth Left-Right Aperture', color='orange')

    # Add labels and legend
    ax.set_xlabel('Frame Index', fontsize=14, fontweight='bold')
    ax.set_ylabel('Aperture Measurement', fontsize=14, fontweight='bold')
    title = f"Eyes and Mouth Aperture Measurements vs. Frame Index"
    ax.set_title(title, fontsize=18, fontweight='bold')
    ax.legend()

    # Add gridlines in soft gray
    ax.grid(color='gray', linestyle=':', linewidth=0.5)

    # Display the plot
    plt.show()

for array in X_train_normalized_padded:
    draw_data(array)
    # time.sleep(10)


NameError: name 'X_train_normalized_padded' is not defined